In [1]:
import pandas as pd
from decouple import config
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta, time
import time 
from sqlalchemy import text
import oracledb
import sys

In [2]:
#CONEXIONES
DB_USER=config("USER2_BD_SDC_POSTGRES")
DB_PASSWORD=config("PASS2_BD_SDC_POSTGRES")
DB_NAME="GCTIC_SDC"
DB_PORT="5432"
DB_HOST=config("HOST2_BD_SDC_POSTGRES")
cadena1  = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine1 = create_engine(cadena1)
connection1 = engine1.connect()

In [3]:
oracledb.init_oracle_client()
oracledb.version = "8.3.0"
sys.modules["cx_Oracle"] = oracledb
un = config("USER4_BDI_POSTGRES")
pw = config("PASS4_BDI_POSTGRES")
hostname=config("HOST4_BDI_POSTGRES")
service_name="WNET"
port = 1527

engine0 = create_engine(f'oracle://{un}:{pw}@',connect_args={
            "host": hostname,
            "port": port,
            "service_name": service_name
        }
    )

connection0 = engine0.connect()

In [4]:
tabla = "citas_cext"
col = "citambproconfec"

fecha_ini = pd.read_sql_query("SELECT fec_ini FROM log_etl where id_proc=1", con=connection1)
fecha_ini= fecha_ini.iloc[0, 0]

fecha_fin = pd.read_sql_query("SELECT fec_ter FROM log_etl where id_proc=1", con=connection1)
fecha_fin= fecha_fin.iloc[0, 0]

dias_por_intervalo = 5

# Inicializa la fecha actual
fecha_actual = fecha_ini

In [5]:
from sqlalchemy import text
import pandas as pd
from datetime import datetime, timedelta

# Supongo que 'fecha_ini' y 'fecha_fin' son datetime objects y 'dias_por_intervalo' es un entero
fecha_actual = fecha_ini

for i in range(0, (fecha_fin - fecha_ini).days + 1, dias_por_intervalo):

    inicioTiempo = time.time()
    now_inicio = datetime.now()

    fecha_ini_intervalo = fecha_actual
    fecha_fin_intervalo = fecha_actual + timedelta(days=dias_por_intervalo - 1)

    fecha_ini_str = fecha_ini_intervalo.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin_intervalo.strftime('%Y-%m-%d')

    print(f"Procesando de {fecha_ini_str} al {fecha_fin_str}")

    now1 = datetime.now()
    query1 = f"UPDATE log_etl SET fec_act ='{now1}' WHERE id_proc=1"
    c1 = text(query1)
    connection1.execute(c1)

    query = f"""
    SELECT
        ras.REDASISCOD,
        ras.REDASISDES,
        cas.CENASICOD,
        cas.CENASIDES,
        aho.AREHOSCOD,
        aho.AREHOSDES,
        sho.SERVHOSCOD,
        sho.SERVHOSDES,
        act.ACTDES,
        act.ACTCOD || ace.ACTESPCOD AS ACTESPCOD,
        ace.ACTESPNOM,
        med.PERNACFEC AS MEDNACFEC,
        med.PERSEXOCOD AS MEDSEXO,
        goc.GRUPOCUPCOD,
        goc.GRUPOCUPNOM,
        cot.CONDTRABCOD,
        cot.CONDTRABNOM,
        pac.PERDOCIDENNUM,
        pac.PERNACFEC AS PACNACFEC,
        pac.PERSEXOCOD AS PACSEXO,
        cit.CITAMBPROCONFEC,
        cit.CITAMBCREFEC,
        cit.CITAMBHORCIT,
        tci.TIPOCITACOD,
        tci.TIPOCITANOM,
        coc.CONDCITACOD,
        coc.CONDCITANOM,
        moc.MODOTORCITACOD,
        moc.MODOTORCITANOM,
        eci.ESTCITCOD,
        eci.ESTCITNOM
    FROM
        CTCAM10 cit
        LEFT OUTER JOIN CMCAS10 cas ON cit.CITAMBCENASICOD = cas.CENASICOD
        LEFT OUTER JOIN CMRAS10 ras ON cas.REDASISCOD = ras.REDASISCOD
        LEFT OUTER JOIN CMAHO10 aho ON cit.CITAMBAREHOSCOD = aho.AREHOSCOD
        LEFT OUTER JOIN CMSHO10 sho ON cit.CITAMBSERVHOSCOD = sho.SERVHOSCOD
        LEFT OUTER JOIN CMACT10 act ON cit.CITAMBACTCOD = act.ACTCOD
        LEFT OUTER JOIN CMACE10 ace ON cit.CITAMBACTCOD = ace.ACTCOD AND cit.CITAMBACTESPCOD = ace.ACTESPCOD
        LEFT OUTER JOIN CMPER10 med ON cit.CITAMBPERASISDOCIDENNUM = med.PERDOCIDENNUM
        LEFT OUTER JOIN CMPRS10 prs ON cit.CITAMBPERASISDOCIDENNUM = prs.PERASISDOCIDENNUM
        LEFT OUTER JOIN CBGOC10 goc ON prs.GRUPOCUPCOD = goc.GRUPOCUPCOD
        LEFT OUTER JOIN CBCOT10 cot ON prs.CONDTRABCOD = cot.CONDTRABCOD
        LEFT OUTER JOIN CBTCI10 tci ON cit.TIPOCITACOD = tci.TIPOCITACOD
        LEFT OUTER JOIN CBCOC10 coc ON cit.CONDCITACOD = coc.CONDCITACOD
        LEFT OUTER JOIN CBMOC10 moc ON cit.MODOTORCITACOD = moc.MODOTORCITACOD
        LEFT OUTER JOIN CBECI10 eci ON cit.ESTCITCOD = eci.ESTCITCOD
        LEFT OUTER JOIN CMPER10 pac ON cit.CITAMBPACSECNUM = pac.PERSECNUM
    WHERE
        act.ACTCOD = '91' 
        AND cit.CITAMBPROCONFEC >= TO_DATE('{fecha_ini_str}', 'YYYY-MM-DD')
        AND cit.CITAMBPROCONFEC < TO_DATE('{fecha_fin_str}', 'YYYY-MM-DD')
    """
    base = pd.read_sql_query(query, con=connection0)
    print("LECTURA FINALIZADA")

    print("Calculando el precedente del paciente")
    # Calcular el precedente del paciente
    base['NUM_CITAS'] = base.groupby('perdocidennum')['perdocidennum'].transform('count')
    base['NUM_DESERCIONES'] = base.groupby('perdocidennum')['estcitnom'].transform(lambda x: (x == 'DESERCION').sum())
    # base['TASA_DESERCION'] = base['NUM_DESERCIONES'] / base['NUM_CITAS']

    #base = base.drop('NUM_CITAS',axis=1)
    #base = base.drop('NUM_DESERCIONES',axis=1)

    #borrando = f"DELETE FROM {tabla} WHERE {col} >= '{fecha_ini_str}' and {col} < '{fecha_fin_str}'"

    #print("BORRANDO PARA ACTUALIZAR")
    #borrado = connection1.execute(borrando)

    print("SUBIENDO DATOS")
    base.to_sql(name=f'{tabla}', con=connection1, if_exists='append', index=False,chunksize=80000)    

    fecha_actual = fecha_fin_intervalo

    finproceso=time.time()
    tiempoproceso=finproceso - inicioTiempo
    tiempoproceso=round(tiempoproceso,3)
    

    print("Proceso completado satisfactoriamente en " + str(tiempoproceso)+" segundos")

Procesando de 2024-01-01 al 2024-01-05
LECTURA FINALIZADA
Calculando el precedente del paciente


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "citas_cext" does not exist
LINE 3:     FROM citas_cext
                 ^

[SQL: 
    SELECT perdocidennum, num_citas, num_deserciones
    FROM citas_cext
    WHERE citambproconfec >= '2024-01-01' and citambproconfec < '2024-01-05'
    ]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [11]:
query2=f"UPDATE log_etl SET fec_ini ='{now2}' WHERE id_proc=1"
c2 = text(query2)
connection1.execute(c2)

connection0.close()
connection1.close()

engine0.dispose()
engine1.dispose()